In [11]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import lightgbm as lgb

In [2]:
# Generar de nuevo el diccionario teniendo en cuenta la diferencia
# entre uint e int
dict_dtypes = {
    'IsBeta': np.int8,                                                              
    'RtpStateBitfield': np.int8,                                                    
    'IsSxsPassiveMode': np.int8,                                                    
    'DefaultBrowsersIdentifier': np.int16,                                          
    'AVProductStatesIdentifier': np.int32,                                          
    'AVProductsInstalled': np.int8,                                                 
    'AVProductsEnabled': np.int8,                                                   
    'HasTpm': np.int8,                                                              
    'CountryIdentifier': np.int8,                                                   
    'CityIdentifier': np.int32,                                                     
    'OrganizationIdentifier': np.int8,
    'GeoNameIdentifier': np.int16,                                                  
    'LocaleEnglishNameIdentifier': np.int16,                                        
    'OsBuild': np.int16,                                                            
    'OsSuite': np.int16,                                                            
    'IsProtected': np.int8,
    'AutoSampleOptIn': np.int8,                                                     
    'SMode': np.int8,                                                               
    'IeVerIdentifier': np.int16,                                                    
    'Firewall': np.int8,                                                            
    'UacLuaenable': np.int32,                                                       
    'Census_OEMNameIdentifier': np.int16,                                           
    'Census_OEMModelIdentifier': np.int32,
    'Census_ProcessorCoreCount': np.int8,                                           
    'Census_ProcessorManufacturerIdentifier': np.int8,
    'Census_ProcessorModelIdentifier': np.int16,                                    
    'Census_PrimaryDiskTotalCapacity': np.int32,                                    
    'Census_SystemVolumeTotalCapacity': np.int32,                                   
    'Census_HasOpticalDiskDrive': np.int8,                                          
    'Census_TotalPhysicalRAM': np.int32,                                            
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float32,                   
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.int16,                  
    'Census_InternalPrimaryDisplayResolutionVertical': np.int16,                    
    'Census_InternalBatteryNumberOfCharges': np.int32,                              
    'Census_OSBuildNumber': np.int16,
    'Census_OSBuildRevision': np.int16,                                             
    'Census_OSInstallLanguageIdentifier': np.int8,                                  
    'Census_OSUILocaleIdentifier': np.int8,                                         
    'Census_IsPortableOperatingSystem': np.int8,                                    
    'Census_IsFlightingInternal': np.int8,                                          
    'Census_IsFlightsDisabled': np.int8,
    'Census_ThresholdOptIn': np.int8,                                               
    'Census_FirmwareManufacturerIdentifier': np.int16,                              
    'Census_FirmwareVersionIdentifier': np.int32,                                   
    'Census_IsSecureBootEnabled': np.int8,                                          
    'Census_IsWIMBootEnabled': np.int8,                                             
    'Census_IsVirtualDevice': np.int8,                                              
    'Census_IsTouchEnabled': np.int8,
    'Census_IsPenCapable': np.int8,                                                 
    'Census_IsAlwaysOnAlwaysConnectedCapable': np.int8,                             
    'Wdft_IsGamer': np.int8,                                                        
    'Wdft_RegionIdentifier': np.int8,                                               
    'HasDetections': np.int8,
    'Census_InternalBatteryType_informed': np.int8,                                 
    'ProductName_index': np.int8,                                                   
    'Platform_index': np.int8,                                                      
    'Processor_index': np.int8,                                                     
    'OsPlatformSubRelease_index': np.int8,                                          
    'SkuEdition_index': np.int8,                                                    
    'PuaMode_index': np.int8,                                                       
    'SmartScreen_index': np.int8,                                                   
    'Census_MDC2FormFactor_index': np.int8,                                         
    'Census_DeviceFamily_index': np.int8,                                           
    'Census_ProcessorClass_index': np.int8,                                         
    'Census_PrimaryDiskTypeName_index': np.int8,                                    
    'Census_ChassisTypeName_index': np.int8,                                        
    'Census_PowerPlatformRoleName_index': np.int8,                                  
    'Census_InternalBatteryType_index': np.int8,                                    
    'Census_OSArchitecture_index': np.int8,                                         
    'Census_OSBranch_index': np.int8,                                               
    'Census_OSEdition_index': np.int8,                                              
    'Census_OSSkuName_index': np.int8,                                              
    'Census_OSInstallTypeName_index': np.int8,                                      
    'Census_OSWUAutoUpdateOptionsName_index': np.int8,                              
    'Census_GenuineStateName_index': np.int8,                                       
    'Census_ActivationChannel_index': np.int8,                                      
    'Census_FlightRing_index': np.int8,                                             
    'Census_OSVersion_0': np.int8,                                                  
    'Census_OSVersion_1': np.int8,                                                  
    'Census_OSVersion_2': np.int16,
    'Census_OSVersion_3': np.int16,                                                 
    'EngineVersion_2': np.int16,                                                    
    'EngineVersion_3': np.int8,                                                     
    'AppVersion_1': np.int8,                                                        
    'AppVersion_2': np.int16,                                                       
    'AppVersion_3': np.int16,                                                       
    'AvSigVersion_0': np.int8,                                                      
    'AvSigVersion_1': np.int16,                                                     
    'AvSigVersion_2': np.int16,                                                     
    'OsVer_0': np.int8,                                                             
    'OsVer_1': np.int8,                                                             
    'OsVer_2': np.int8,                                                             
    'OsVer_3': np.int8,                                                             
    'OsBuildLab_0': np.int16,                                                       
    'OsBuildLab_1': np.int16,                                                       
    'OsBuildLab_4_0': np.int32,                                                     
    'OsBuildLab_4_1': np.int16,                                                     
    'OsBuildLab_2_index': np.int8,                                                  
    'OsBuildLab_3_index': np.int8
}

In [3]:
path = '../data/train_final_0'

In [15]:
# import os
# import pandas as pd 
# from multiprocessing import Pool

# # wrap your csv importer in a function that can be mapped
# def read_csv(filename):
#     'converts a filename to a pandas dataframe'
#     return pd.read_csv(filename)


# def readCsv(processes=8, path=''):
#     file_list = glob.glob(path + "/*.csv")
#     # set up your pool
#     pool = Pool(processes=processes) # or whatever your hardware can support

#     # get a list of file names
#     files = os.listdir(path)

#     # have your pool map the file names to dataframes
#     df_list = pool.map(read_csv, file_list)

#     # reduce the list of dataframes to a single dataframe
#     combined_df = pd.concat(df_list, ignore_index=True)
    
#     return combined_df


In [ ]:
# a = readCsv(path = path)

In [4]:
allFiles = glob.glob(path + "/*.csv")

list_ = []
n = 0
for file_ in allFiles:
    df = pd.read_csv(file_, dtype=dict_dtypes)
    list_.append(df)
    if n%10==0:
        print(n)
    n+=1

frame = pd.concat(list_, axis = 0, ignore_index = True)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [5]:
frame.shape

(8921483, 100)

In [6]:
frame.memory_usage(deep=True).sum()/1024**2

2075.9982414245605

In [7]:
train = frame

In [8]:
train.head()

,MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,OsVer_0,OsVer_1,OsVer_2,OsVer_3,OsBuildLab_0,OsBuildLab_1,OsBuildLab_4_0,OsBuildLab_4_1,OsBuildLab_2_index,OsBuildLab_3_index
0,00005c4f5d93f600a61b41581e5b66b2,0,7,0,0,53447,1,1,1,107,...,10,0,0,0,17134,1,180410,1804,0,0
1,000122bd82d8f08048877cf4c3ea0396,0,7,0,0,53447,1,1,1,60,...,10,0,0,0,17134,1,180410,1804,0,0
2,000171542375a3bc2f87e14f135f671f,0,7,0,0,53447,1,1,1,-87,...,10,0,0,0,10240,17443,170602,2340,0,6
3,00017c8e5572b1ac9714bec63539f326,0,7,0,0,53447,1,1,1,43,...,10,0,0,0,17134,1,180410,1804,0,0
4,00037a5ff54ba7297fe33b5fc84828c2,0,7,0,0,53447,1,1,1,-51,...,10,0,0,0,16299,637,180808,1748,0,7


In [78]:
train_mini = train.iloc[:700000,:]

In [79]:
sel_cols = [c for c in train.columns if c not in ['MachineIdentifier', 'HasDetections']]

In [80]:
X = train_mini.loc[:, sel_cols]
y = train_mini.loc[:,'HasDetections']

In [99]:
params = {
    'objective' :['binary'],
    'learning_rate' : [0.05],
    'num_leaves' : [31],
    'feature_fraction': [1], 
    'bagging_fraction': [0.8], 
    'bagging_freq':[1],
    'boosting_type' : ['gbdt'],
    'metric': ['binary_error'],
    'max_depth': [10]
}

In [100]:
mdl_lgb = lgb.LGBMClassifier(n_jobs=4)

In [101]:
from sklearn.model_selection import GridSearchCV

In [102]:
lgb_model = GridSearchCV(estimator=mdl_lgb,
                                     param_grid=params,
                                     n_jobs=4,
                                     cv=5,
                                     verbose=5,
                                     scoring='roc_auc')

In [103]:
lgb_model.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  2.2min remaining:  3.3min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  2.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=4, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'objective': ['binary'], 'learning_rate': [0.05], 'num_leaves': [31], 'feature_fraction': [1], 'bagging_fraction': [0.8], 'bagging_freq': [1], 'boosting_type': ['gbdt'], 'metric': ['binary_error'], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=5)

In [104]:
lgb_model.cv_results_

/home/errodringer/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/errodringer/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/errodringer/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/errodringer/.local/

{'mean_fit_time': array([107.68031874]),
 'std_fit_time': array([45.40841671]),
 'mean_score_time': array([0.75657396]),
 'std_score_time': array([0.19810851]),
 'param_bagging_fraction': masked_array(data=[0.8],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_bagging_freq': masked_array(data=[1],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_boosting_type': masked_array(data=['gbdt'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_feature_fraction': masked_array(data=[1],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.05],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[10],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['binary_error'],
            

In [48]:
mdl_lgb.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=4, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [49]:
mdl_lgb.feature_importances_

array([  0,  42,  14,  21, 258,  69,  29,   1,  95,  19,   8,  28,  80,
        17,   9,  11,   0,  26,  36,   8,   7,  26,  20,  16,   3,  59,
        62,  59,  18,  53,  71,  26,  25,  42,  11,  82,  38,  62,   0,
         0,   0,  17,  23,  17,  15,   4,  37,   7,   3,   3, 100,  99,
         0,   0,   1,  15,  12,   9,   9, 151,  12,   0,   8,   1,  20,
        10,   0,   0,  21,  32,  13, 119,  17,  30,  56,  14,   0,   0,
         0,   2,  65,  18,  15, 110, 125,   0, 107, 131,   0,   0,   0,
         0,   0,  29,  60,   6,   1,   5])

In [55]:
dict_importances = dict()
for i, j in zip(sel_cols, mdl_lgb.feature_importances_):
    dict_importances[i] = j

In [64]:
df_imp = pd.DataFrame({'col': sel_cols, 'imp': mdl_lgb.feature_importances_})

In [65]:
df_imp.sort_values('imp', ascending=False)

,col,imp
4,AVProductStatesIdentifier,258
59,SmartScreen_index,151
87,AvSigVersion_2,131
84,AppVersion_3,125
71,Census_OSInstallTypeName_index,119
83,AppVersion_2,110
86,AvSigVersion_1,107
50,Wdft_IsGamer,100
51,Wdft_RegionIdentifier,99
8,CountryIdentifier,95


In [73]:
col_sel_2 = df_imp.loc[:35, 'col']

In [74]:
col_sel_2

0                                                IsBeta
1                                      RtpStateBitfield
2                                      IsSxsPassiveMode
3                             DefaultBrowsersIdentifier
4                             AVProductStatesIdentifier
5                                   AVProductsInstalled
6                                     AVProductsEnabled
7                                                HasTpm
8                                     CountryIdentifier
9                                        CityIdentifier
10                               OrganizationIdentifier
11                                    GeoNameIdentifier
12                          LocaleEnglishNameIdentifier
13                                              OsBuild
14                                              OsSuite
15                                          IsProtected
16                                      AutoSampleOptIn
17                                              